In [40]:
import comet_ml
# import comet_ml in the top of your file
from comet_ml import Experiment
    
# Add the following code anywhere in your machine learning file
experiment = Experiment(api_key="pFR0ZDxiYAyt5B7JfZiEjBqfB",
                        project_name="unsupervised-rm2", workspace="team-jhb-ss3",log_code=True)

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/thavha2/movie-recommender-systems/c57cae925ee0485b8fdcd4bb4427ca3e
COMET INFO:   Uploads:
COMET INFO:     environment details      : 1
COMET INFO:     filename                 : 1
COMET INFO:     git metadata             : 1
COMET INFO:     git-patch (uncompressed) : 1 (26 MB)
COMET INFO:     installed packages       : 1
COMET INFO: ---------------------------
COMET INFO: old comet version (3.1.11) detected. current: 3.1.14 please update your comet lib with command: `pip install --no-cache-dir --upgrade comet_ml`
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/team-jhb-ss3/unsupervised-rm2/11d24e5040cd4e85a01043ca934dd0ec



In [41]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud

In [42]:
train = pd.read_csv(r"C:\Users\Dell\Documents\Dev\EDSA\Unsupervised_learning\train.csv")
train.head()

,userId,movieId,rating,timestamp
0,5163,57669,4.000,1518349992
1,106343,5,4.500,1206238739
2,146790,5459,5.000,1076215539
3,106362,32296,2.000,1423042565
4,9041,366,3.000,833375837


In [43]:
movies = pd.read_csv(r"C:\Users\Dell\Documents\Dev\EDSA\Unsupervised_learning\movies.csv")
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [44]:
df = pd.merge(train,movies, on='movieId')

In [45]:
df.drop(['genres'], axis =1, inplace=True)

In [46]:
df = df.dropna(axis=0, subset=['title'])
rating_count = (df.groupby(by=['title'])['rating'].count().reset_index().rename(columns={'rating':'totalRatingCount'})[['title','totalRatingCount']])
rating_count.head()

,title,totalRatingCount
0,"""BLOW THE NIGHT!"" Let's Spend the Night Togeth...",1
1,"""Great Performances"" Cats (1998)",67
2,#1 Cheerleader Camp (2010),5
3,#Female Pleasure (2018),1
4,#FollowMe (2019),5


In [47]:
rating_with_totalRatingCount = df.merge(rating_count,left_on='title',right_on='title', how='left')
rating_with_totalRatingCount.head()

,userId,movieId,rating,timestamp,title,totalRatingCount
0,5163,57669,4.000,1518349992,In Bruges (2008),4253
1,87388,57669,3.500,1237455297,In Bruges (2008),4253
2,137050,57669,4.000,1425631854,In Bruges (2008),4253
3,120490,57669,4.500,1408228517,In Bruges (2008),4253
4,50616,57669,4.500,1446941640,In Bruges (2008),4253


In [48]:
pd.set_option('display.float_format' ,lambda x: '%.3f' % x)
print(rating_count['totalRatingCount'].describe())

count   48151.000
mean      207.681
std      1093.466
min         1.000
25%         1.000
50%         4.000
75%        26.000
max     32831.000
Name: totalRatingCount, dtype: float64


In [49]:
data = train

In [50]:
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import KFold
from surprise.model_selection import cross_validate
from surprise import NormalPredictor
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import KNNBaseline
from surprise import SVD
from surprise import SVDpp
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering
from surprise.accuracy import rmse
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV
from collections import defaultdict

In [51]:
ratings_dict = {'itemID': list(train.movieId),
                'userID': list(train.userId),
                'rating': list(train.rating)}

df = pd.DataFrame(ratings_dict)
df.shape

(10000038, 3)

In [52]:

reader = Reader(line_format='user item rating timestamp', sep='\t')

reader = Reader(rating_scale=(0.5, 5.0))

data = Dataset.load_from_df(df[['userID', 'itemID', 'rating']], reader)


In [53]:
from surprise.model_selection import train_test_split

from surprise.model_selection import KFold

kf = KFold(n_splits=10)
kf.split(data)
algo = SVD(n_factors=35, n_epochs=25, lr_all=0.008, reg_all=0.08)
trainset = data.build_full_trainset()
algo.fit(trainset)

KeyboardInterrupt: 

In [ ]:
experiment.log_model('SVD',algo)

In [ ]:
test = pd.read_csv(r"C:\Users\Dell\Documents\Dev\EDSA\Unsupervised_learning\test.csv")
test.head()

In [ ]:
test = test.rename(columns={'movieId': 'itemId'})

In [ ]:
test['Id'] = test[['userId', 'itemId']].apply(tuple, axis=1)

In [ ]:
test['rating'] = test['Id'].apply(lambda x: algo.predict(x[0],x[1])[3])

In [ ]:
test['Id'] = test['Id'].apply(lambda x: str(x[0])+'_'+ str(x[1]))

In [ ]:
sub = test[['Id','rating']]

In [ ]:
sub.to_csv('submission.csv', index=False)

In [ ]:
experiment.end()